In [1]:
import pandas                   as     pd
import numpy                    as     np
import seaborn                  as     sns
import matplotlib.pyplot        as     plt
import statsmodels.api          as     sm
from   sklearn.preprocessing    import OrdinalEncoder
from   sklearn                  import metrics
from   sklearn                  import tree
import sys, os
import os
os.chdir(r'D:\DrPKV\20220519')

In [2]:
import time
from    datetime   import datetime
from    datetime   import timedelta

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Multiple Imputation by Chained Equation.


#### Multiple Imputation by Chained Equation assumes that data is MAR, i.e. missing at random.


We observe that on a few occasions, data missing in a dataset and is related to the other features and hence they can be predicted using other feature values.

Ref: https://www.numpyninja.com/post/mice-and-knn-missing-value-imputations-through-python

It cannot be imputed with general ways of using mean, mode, or median which might pollute the data. 

In [4]:
### Imputing missing numerical values using the IterativeImputer class in sklearn 

from sklearn.experimental  import   enable_iterative_imputer
from sklearn.impute        import   IterativeImputer
from sklearn.linear_model   import   LinearRegression

### https://www.numpyninja.com/post/mice-and-knn-missing-value-imputations-through-python

In [5]:
file_name = './Data/BankruptcyData.xlsx'

In [6]:
df     =   pd.read_excel(file_name, sheet_name = 'Full data')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Row                                       9000 non-null   int64  
 1   Year                                      9000 non-null   int64  
 2   Company_name                              9000 non-null   object 
 3   Year Encoded                              9000 non-null   int64  
 4    Dummy Coded :Healthy=1; NPA= 0           9000 non-null   int64  
 5   Asset_turnover                            8299 non-null   float64
 6   Receivable_turnover(new)                  7390 non-null   float64
 7   Inventory_turnover                        5702 non-null   float64
 8   Cash_ratio                                8278 non-null   float64
 9   Quick_ratio                               8278 non-null   float64
 10  Current_ratio                       

In [8]:
df.columns

Index(['Row', 'Year', 'Company_name', 'Year Encoded',
       ' Dummy Coded :Healthy=1; NPA= 0', 'Asset_turnover',
       'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio',
       'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)',
       'ROI(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder's Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders' funds',
       'Shareholderquity_code'],
      dtype='object')

### Convert categorical column to numeric

https://pbpython.com/categorical-encoding.html

In [9]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero & Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero & Missing Values'] = 100 * mz_table['Total Zero & Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [10]:
mdf =  missing_zero_values_table(df)
mdf.shape

Your selected dataframe has 34 columns and 9000 Rows.
There are 29 columns that have missing values.


(29, 6)

In [11]:
mdf.index

Index(['ROI(new)', 'Shareholderquity_code', 'Inventory_turnover',
       'Receivable_turnover(new)', 'Interest_coverage',
       'Operating Cash Flow/Total Debt', 'Asset_coverage', 'ROCE_CE',
       'Fixed Asset Turnover Ratio', 'Cash_ratio', 'Current_ratio',
       'Quick_ratio', 'Sales_CE', 'Asset_turnover',
       'Operating Cash Flow/Shareholder's Equity', 'ROE(new)', 'debt_equity',
       'debt_asset', 'ROA(new)', 'YOY Sales Growth Rate', 'EBIT_Sales',
       'ROS(new)', 'Operating Cash Flow/Total Sales', 'YOY EBIT Growth Rate',
       'debt_income', 'Total shareholders' funds', 'Grossvaluedadded',
       'Ln_GVA', 'Changeinsales_Industry'],
      dtype='object')

In [12]:
mdf

,Zero Values,Missing Values,% of Total Values,Total Zero & Missing Values,% Total Zero & Missing Values,Data Type
ROI(new),25,4132,45.90000,4157,46.20000,float64
Shareholderquity_code,3305,3790,42.10000,7095,78.80000,float64
Inventory_turnover,4364,3298,36.60000,7662,85.10000,float64
Receivable_turnover(new),401,1610,17.90000,2011,22.30000,float64
Interest_coverage,218,1569,17.40000,1787,19.90000,float64
Operating Cash Flow/Total Debt,15,1306,14.50000,1321,14.70000,float64
Asset_coverage,0,1306,14.50000,1306,14.50000,float64
ROCE_CE,70,1258,14.00000,1328,14.80000,float64
Fixed Asset Turnover Ratio,476,1007,11.20000,1483,16.50000,float64
Cash_ratio,21,722,8.00000,743,8.30000,float64


### Drop variables having more than 40% of missing values
1. ROI(new)
2. Shareholderquity_code

### The following variables,all numerical type are having missing values

*  1) 'Inventory_turnover'
*  2) 'Receivable_turnover(new)'
*  3) 'Interest_coverage'
*  4) 'Asset_coverage'
*  5) 'Operating Cash Flow/Total Debt'
*  6) 'ROCE_CE'
*  7) 'Fixed Asset Turnover Ratio'
*  8) 'Cash_ratio'
*  9) 'Quick_ratio'
* 10) 'Current_ratio'
* 11) 'Sales_CE'
* 12) 'Asset_turnover'
* 13) 'debt_equity'
* 14) 'ROE(new)'
* 15) "Operating Cash Flow/Shareholder's Equity"
* 16) 'debt_asset'
* 17) 'ROA(new)'
* 18) 'YOY Sales Growth Rate'
* 19) 'EBIT_Sales'
* 20) 'ROS(new)'
* 21) 'Operating Cash Flow/Total Sales'
* 22) 'YOY EBIT Growth Rate' 
* 23) 'debt_income'
* 24) "Total shareholders' funds"
* 25) 'Changeinsales_Industry'
* 26) 'Grossvaluedadded'
* 27) 'Ln_GVA'

In [13]:
print("\nBefore removing null values \n Rows %d Columns %d" % (df.shape[0], df.shape[1]))
df.drop(['ROI(new)','Shareholderquity_code' ], axis = 1, inplace = True)
print("\nAfter removing null values \n Rows %d Columns %d" % (df.shape[0], df.shape[1]))


Before removing null values 
 Rows 9000 Columns 34

After removing null values 
 Rows 9000 Columns 32


In [14]:
num_variables_df = df.select_dtypes(include = 'number')
print(num_variables_df.shape)
print(num_variables_df.columns)

(9000, 31)
Index(['Row', 'Year', 'Year Encoded', ' Dummy Coded :Healthy=1; NPA= 0',
       'Asset_turnover', 'Receivable_turnover(new)', 'Inventory_turnover',
       'Cash_ratio', 'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)',
       'ROS(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder's Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders' funds'],
      dtype='object')


In [15]:
lreg         =  LinearRegression()
imp          =  IterativeImputer(estimator= lreg, missing_values = np.nan, max_iter = 10, verbose = 2,\
                                 imputation_order= 'roman',random_state = 0)
X            =  imp.fit_transform(num_variables_df)

[IterativeImputer] Completing matrix with shape (9000, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.47
[IterativeImputer] Change: 4378730442.761648, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.78
[IterativeImputer] Change: 2741521048.0803456, scaled tolerance: 350030860.0 
[IterativeImputer] Ending imputation round 3/10, elapsed time 1.07
[IterativeImputer] Change: 108970542.8637385, scaled tolerance: 350030860.0 
[IterativeImputer] Early stopping criterion reached.


In [16]:
X_df           =    pd.DataFrame(X, columns = ['Row', 'Year', 'Year Encoded', 'Dummy Coded :Healthy=1; NPA= 0',
       'Asset_turnover', 'Receivable_turnover(new)', 'Inventory_turnover',
       'Cash_ratio', 'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)',
       'ROS(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder"s Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders" funds'])
missing_zero_values_table(X_df)  

Your selected dataframe has 31 columns and 9000 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero & Missing Values,% Total Zero & Missing Values,Data Type


In [17]:
X_df['Row']                                  =  X_df['Row'].astype(int)
X_df['Year']                                 =  X_df['Year'].astype(int)
X_df['Year Encoded']                         =  X_df['Year Encoded'].astype(int)
X_df['Dummy Coded :Healthy=1; NPA= 0']       =  X_df['Dummy Coded :Healthy=1; NPA= 0'].astype(int)

In [18]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 31 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Row                                       9000 non-null   int32  
 1   Year                                      9000 non-null   int32  
 2   Year Encoded                              9000 non-null   int32  
 3   Dummy Coded :Healthy=1; NPA= 0            9000 non-null   int32  
 4   Asset_turnover                            9000 non-null   float64
 5   Receivable_turnover(new)                  9000 non-null   float64
 6   Inventory_turnover                        9000 non-null   float64
 7   Cash_ratio                                9000 non-null   float64
 8   Quick_ratio                               9000 non-null   float64
 9   Current_ratio                             9000 non-null   float64
 10  ROA(new)                            

In [19]:
X_df.isnull().sum()

Row                                         0
Year                                        0
Year Encoded                                0
Dummy Coded :Healthy=1; NPA= 0              0
Asset_turnover                              0
Receivable_turnover(new)                    0
Inventory_turnover                          0
Cash_ratio                                  0
Quick_ratio                                 0
Current_ratio                               0
ROA(new)                                    0
ROE(new)                                    0
ROS(new)                                    0
debt_asset                                  0
debt_equity                                 0
debt_income                                 0
Interest_coverage                           0
Asset_coverage                              0
EBIT_Sales                                  0
Sales_CE                                    0
ROCE_CE                                     0
Changeinsales_Industry            

### Convert categorical column to numeric column

In [20]:
ord_enc             = OrdinalEncoder()
df["Company Code"]  = ord_enc.fit_transform(df[["Company_name"]])
df[["Company Code", "Company_name"]].head(11)

,Company Code,Company_name
0,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
1,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
2,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
3,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
4,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
5,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
6,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
7,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
8,51.00000,ASCO NUMATICS (INDIA) PRIVATE LIMITED - Asco N...
9,52.00000,ASCOT HOTELS AND RESORTS PRIVATE LIMITED - Asc...


### Retain the numeric column, Company Code and drop the column Company_name

In [21]:
df.drop(['Company_name'], axis = 1, inplace = True)

### Rename the column " Dummy Coded \:Healthy=1; NPA= 0" as "Target"

In [22]:
df.rename({"Dummy Coded \:Healthy=1; NPA= 0" : 'Target'}, inplace = True)

In [23]:
collist  = [ "Row", "Year", "Company_name", "Year Encoded",\
"Target", "Asset_turnover",\
"Receivable_turnover(new)", "Inventory_turnover", "Cash_ratio",\
"Quick_ratio", "Current_ratio", "ROA(new)", "ROE(new)", "ROS(new)",\
 "debt_asset", "debt_equity", "debt_income",\
"Interest_coverage", "Asset_coverage", "EBIT_Sales", "Sales_CE",\
"ROCE_CE", "Changeinsales_Industry", "Grossvaluedadded", "Ln_GVA",\
"Operating Cash Flow/Total Sales", "Operating Cash Flow/Total Debt",\
"Operating Cash Flow/Shareholder's Equity",\
"Fixed Asset Turnover Ratio", "YOY Sales Growth Rate",\
"YOY EBIT Growth Rate", "Total shareholders' funds"]

In [24]:
df.columns = collist

In [25]:
print(collist);print(len(collist))

['Row', 'Year', 'Company_name', 'Year Encoded', 'Target', 'Asset_turnover', 'Receivable_turnover(new)', 'Inventory_turnover', 'Cash_ratio', 'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)', 'ROS(new)', 'debt_asset', 'debt_equity', 'debt_income', 'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE', 'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA', 'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt', "Operating Cash Flow/Shareholder's Equity", 'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate', 'YOY EBIT Growth Rate', "Total shareholders' funds"]
32


In [26]:
collist1 =  X_df.columns

In [27]:
len(collist1)

31

In [28]:
collist1

Index(['Row', 'Year', 'Year Encoded', 'Dummy Coded :Healthy=1; NPA= 0',
       'Asset_turnover', 'Receivable_turnover(new)', 'Inventory_turnover',
       'Cash_ratio', 'Quick_ratio', 'Current_ratio', 'ROA(new)', 'ROE(new)',
       'ROS(new)', 'debt_asset', 'debt_equity', 'debt_income',
       'Interest_coverage', 'Asset_coverage', 'EBIT_Sales', 'Sales_CE',
       'ROCE_CE', 'Changeinsales_Industry', 'Grossvaluedadded', 'Ln_GVA',
       'Operating Cash Flow/Total Sales', 'Operating Cash Flow/Total Debt',
       'Operating Cash Flow/Shareholder"s Equity',
       'Fixed Asset Turnover Ratio', 'YOY Sales Growth Rate',
       'YOY EBIT Growth Rate', 'Total shareholders" funds'],
      dtype='object')

In [29]:
diff1 =  set(collist) - set(collist1)
print(diff1)

{"Operating Cash Flow/Shareholder's Equity", "Total shareholders' funds", 'Target', 'Company_name'}


In [30]:
new_df      =    X_df.copy()

In [31]:
new_df['Company_name']  =  df['Company_name']

In [32]:
new_df.head().T

,0,1,2,3,4
Row,1.00000,1.00000,1.00000,1.00000,1.00000
Year,2010.00000,2011.00000,2012.00000,2013.00000,2014.00000
Year Encoded,-8.00000,-7.00000,-6.00000,-5.00000,-4.00000
Dummy Coded :Healthy=1; NPA= 0,1.00000,1.00000,1.00000,1.00000,1.00000
Asset_turnover,0.00000,0.75666,1.07063,1.18632,1.42893
Receivable_turnover(new),0.00000,3.46077,3.95393,4.54417,6.10799
Inventory_turnover,0.00000,0.00000,0.00000,0.00000,0.00000
Cash_ratio,2.01764,1.55843,0.85337,0.96842,1.21196
Quick_ratio,1.55202,1.08407,0.38471,0.37537,0.61504
Current_ratio,3.73717,3.03393,2.56104,2.98033,3.18905


In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 32 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Row                                       9000 non-null   int32  
 1   Year                                      9000 non-null   int32  
 2   Year Encoded                              9000 non-null   int32  
 3   Dummy Coded :Healthy=1; NPA= 0            9000 non-null   int32  
 4   Asset_turnover                            9000 non-null   float64
 5   Receivable_turnover(new)                  9000 non-null   float64
 6   Inventory_turnover                        9000 non-null   float64
 7   Cash_ratio                                9000 non-null   float64
 8   Quick_ratio                               9000 non-null   float64
 9   Current_ratio                             9000 non-null   float64
 10  ROA(new)                            

In [34]:
output_file_name =  './Output/Bankruptcy-data-' + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + '.csv'
new_df.to_csv(output_file_name, index = False)

### END